In [0]:
from sklearn import metrics
import pandas as pd
import numpy as np
import glob
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
import time
import collections
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import defaultdict


In [5]:
from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive/My\ Drive/artigo_plantas
!ls

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).
/gdrive/My Drive/artigo_plantas
resnet50_features.csv


In [0]:
#Variáveis do Dataset de Resultados
arquivo_csv = []
classifier = []
accuracy = []
std_accuracy = []

# Metrics Macro
precision_macro = []
std_precision_macro = []
recall_macro = []
std_recall_macro = []
f1_macro = []
std_f1_macro = []

# Metrics Micro
precision_micro = []
std_precision_micro = []
recall_micro = []
std_recall_micro = []
f1_micro = []
std_f1_micro = []

# Metrics Weighted
precision_weighted = []
std_precision_weighted = []
recall_weighted = []
std_recall_weighted = []
f1_weighted = []
std_f1_weighted = []

# Informações do PCA
n_samples = []
princ_compo = []
variancia = []


In [0]:
def multi_clf_metrics(arq_csv):

	# Cria X (features) e y (labels)
  especies = pd.read_csv(arq_csv, low_memory=False)
	
  if arq_csv == 'plantclef2013.csv' or arq_csv == 'plantclef2013preclustering.csv':
    X = np.array(especies.iloc[2:,0:2048])
    y = np.array(especies.iloc[2:,2048])
  elif arq_csv == 'resnet50_features.csv' or arq_csv == 'resnet152v2_features.csv':
    X = np.array(especies.iloc[0:,0:2048])
    y = np.array(especies.iloc[0:,2048])		
  # Executa o PCA
  pca = PCA(.90)
  principal_components = pca.fit_transform(X)

  classifiers = {'SVC - (linear, ovr)': OneVsRestClassifier(svm.SVC(kernel='linear', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=200,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None)),
					'SVC - (linear, ovo)': svm.SVC(kernel='linear', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=200,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovo', random_state=None),
					'SVC - (polynomial, ovr)': OneVsRestClassifier(svm.SVC(kernel='poly', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=200,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None)),
					'SVC - (polynomial, ovo)': svm.SVC(kernel='poly', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=200,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovo', random_state=None),
					'SVC - (sigmoid, ovr)': OneVsRestClassifier(svm.SVC(kernel='sigmoid', degree=3,coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=200,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None)),
					'SVC - (sigmoid, ovo)': svm.SVC(kernel='sigmoid', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=200,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovo', random_state=None)}

  scoring = ['accuracy',
				'precision_macro','recall_macro','f1_macro',
				'precision_micro','recall_micro','f1_micro',
				'precision_weighted','recall_weighted','f1_weighted']
  # Parametros para rodar com o grid search
  parameters = {'C' : [0.001, 0.01, 0.1, 1, 5, 10, 25, 50, 100, 1000],'gamma':[0.0001, 0.001, 0.01, 0.1, 1]}
  result = []
  for name_clf, clf in classifiers.items():
    print(f'Executando classificador \033[1;31m{name_clf}\033[m')
    classifier.append(name_clf)
    arquivo_csv.append(arq_csv[:-4])
    # GridSearch para achar a melhor combinação de parâmetros (n_jobs=-1 para rodar em paralelo)
    scores = GridSearchCV(clf, parameters,cv=2,return_train_score=False, scoring=scoring, n_jobs=-1)
    scores.fit(principal_components,y)
    result.append(scores)
    print(scores)
  return result

In [8]:
result = multi_clf_metrics('resnet50_features.csv')

Executando classificador SVC - (linear, ovr)


ValueError: ignored